In [1]:
import numpy as np
from scipy.ndimage import gaussian_filter
    
    
def ma(score_list, k=3):
    result=[]
    for score in score_list:
        ma_score=[np.mean(score[:2*i+1]) for i in range(k)]+[np.mean(score[i-k:i+k+1]) for i in range(k,len(score)-k)]+[np.mean(score[-len(score)+2*i+1:]) for i in range(len(score)-k,len(score))]
        result.append(np.array(ma_score))
    return result

def softmax(raw_score, T=1):
    exp_s = np.exp((raw_score - raw_score.max(axis=-1)[..., None])*T)
    sum_s = exp_s.sum(axis=-1)
    return exp_s / sum_s[..., None]

def gen_prop(scores, stride=4, num_frame=32, minimum_len=0):
    frm_duration = len(scores)
    topk_cls = [0]
    topk_labels = label_frame_by_threshold(scores, topk_cls, bw=None, thresh=[0.01, 0.05, 0.1, .15, 0.25, .4, .5, .6, .7, .8, .9, .95, ], multicrop=False)

    bboxes = []
    tol_lst = [0.05, .1, .2, .3, .4, .5, .6, 0.8, 1.0]

    bboxes.extend(build_box_by_search(topk_labels, np.array(tol_lst)))


    # print len(bboxes)
    bboxes = temporal_nms_fallback(bboxes, 0.9)

#     pr_box = [(x[0] / float(frm_duration) * v.duration, x[1] / float(frm_duration) * v.duration) for x in bboxes]

    pr_box = [(x[0]*stride, (x[1]-2)*stride + num_frame) for x in bboxes]

    # filter out too short proposals
    pr_box = list(filter(lambda b: b[1] - b[0] > minimum_len, pr_box))
    return pr_box, [x[3] for x in bboxes]

def label_frame_by_threshold(score_mat, cls_lst, bw=None, thresh=list([0.05]), multicrop=True):
    """
    Build frame labels by thresholding the foreground class responses
    :param score_mat:
    :param cls_lst:
    :param bw:
    :param thresh:
    :param multicrop:
    :return:
    """
    if multicrop:
        f_score = score_mat.mean(axis=1)
    else:
        f_score = score_mat

#     ss = softmax(f_score)
    ss=f_score

    rst = []
    for cls in cls_lst:
        cls_score = ss if bw is None else gaussian_filter(ss, bw)
        for th in thresh:
            rst.append((cls, cls_score > th, f_score))

    return rst

def build_box_by_search(frm_label_lst, tol, min=1):
    boxes = []
    for cls, frm_labels, frm_scores in frm_label_lst:
        length = len(frm_labels)
        diff = np.empty(length+1)
        diff[1:-1] = frm_labels[1:].astype(int) - frm_labels[:-1].astype(int)
        diff[0] = float(frm_labels[0])
        diff[length] = 0 - float(frm_labels[-1])
        cs = np.cumsum(1 - frm_labels)
        offset = np.arange(0, length, 1)

        up = np.nonzero(diff == 1)[0]
        down = np.nonzero(diff == -1)[0]

        assert len(up) == len(down), "{} != {}".format(len(up), len(down))
        for i, t in enumerate(tol):
            signal = cs - t * offset
            for x in range(len(up)):
                s = signal[up[x]]
                for y in range(x + 1, len(up)):
                    if y < len(down) and signal[up[y]] > s:
                        boxes.append((up[x], down[y-1]+1, cls, sum(frm_scores[up[x]:down[y-1]+1])))
                        break
                else:
                    boxes.append((up[x], down[-1] + 1, cls, sum(frm_scores[up[x]:down[-1] + 1])))

            for x in range(len(down) - 1, -1, -1):
                s = signal[down[x]] if down[x] < length else signal[-1] - t
                for y in range(x - 1, -1, -1):
                    if y >= 0 and signal[down[y]] < s:
                        boxes.append((up[y+1], down[x] + 1, cls, sum(frm_scores[up[y+1]:down[x] + 1])))
                        break
                else:
                    boxes.append((up[0], down[x] + 1, cls, sum(frm_scores[0:down[x]+1 + 1])))

    return boxes

def temporal_nms_fallback(bboxes, thresh, score_ind=3):
    """
    One-dimensional non-maximal suppression
    :param bboxes: [[st, ed, cls, score], ...]
    :param thresh:
    :return:
    """
    t1 = np.array([x[0] for x in bboxes])
    t2 = np.array([x[1] for x in bboxes])
    scores = np.array([x[score_ind] for x in bboxes])

    durations = t2 - t1 + 1
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        tt1 = np.maximum(t1[i], t1[order[1:]])
        tt2 = np.minimum(t2[i], t2[order[1:]])
        intersection = tt2 - tt1 + 1
        IoU = intersection / (durations[i] + durations[order[1:]] - intersection).astype(float)

        inds = np.where(IoU <= thresh)[0]
        order = order[inds + 1]

    return [bboxes[i] for i in keep]

In [2]:
score_list = [np.array([0,0,0,0,0,0,0,0,0,0,0,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0.8,0,0,0,0,0,0,0,0,0,0,0,0,0,0.9,0.9,0.9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]),
              np.array([0,0.4,0.5]),
              np.array([0,0.2,0.3,0.5,0.6])]
score_list_ma = ma(score_list)
print('original score')
for scores in score_list:
    print(gen_prop(scores))
print('moving average')
for scores in score_list_ma:
    print(gen_prop(scores))

original score
([(136, 176), (44, 176), (44, 112)], [10.7, 10.7, 7.999999999999999])
([(8, 40), (4, 40)], [0.9, 0.9])
([(16, 48), (4, 48), (12, 48), (8, 48)], [1.6, 1.6, 1.6, 1.6])
moving average
([(36, 184), (32, 124), (40, 116), (36, 120), (44, 112), (48, 108), (52, 104), (56, 100), (124, 188), (128, 184)], [10.700000000000003, 7.999999999999999, 7.999999999999999, 7.999999999999999, 7.885714285714285, 7.657142857142857, 7.314285714285714, 6.857142857142857, 2.7, 2.5714285714285716])
([(12, 44), (4, 52), (12, 52), (20, 52)], [1.85, 1.85, 1.85, 0.5])
([(20, 52), (4, 52), (16, 52), (8, 52)], [1.873333333333333, 1.873333333333333, 1.873333333333333, 1.873333333333333])
